In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!unzip pchdata.zip -d /content/

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
path = '/content/pchdata'
folders = os.listdir(path)

types = []

for folder in folders:
    json_path = os.path.join(path, folder, "GT.json")

    with open(json_path, "r") as f:
        sim_data = json.load(f)
        if sim_data['SimulationInputs']['sigma_bS1']:
          if sim_data['SimulationInputs']['sigma_bS2'] and sim_data['SimulationInputs']['sigma_bS3']:
            types.append(3)
          elif sim_data['SimulationInputs']['sigma_bS2']:
            types.append(2)
          else:
            types.append(1)
        else:
          types.append(-1)
types = np.array(types)

In [ ]:
amplitudes = []
for folder in folders:
    json_path = os.path.join(path, folder, "GT.json")

    with open(json_path, "r") as f:
        sim_data = json.load(f)
          
        if not sim_data['SimulationInputs']['alpha']:
          if sim_data['Amplitudes']['AmpS1']:
            amplitudes.append(sim_data['Amplitudes']['AmpS1'])
          if sim_data['Amplitudes']['AmpS2']:
            amplitudes.append(sim_data['Amplitudes']['AmpS2'])
          if sim_data['Amplitudes']['AmpS3']:
            amplitudes.append(sim_data['Amplitudes']['AmpS3'])
          
amplitudes = np.array(amplitudes)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns

# ---------- Helper: histogram that draws on a provided axes ----------
def publication_hist_ax(ax, amplitudes, title="Amplitude Distribution", log_bins=True, density=False):
    x = np.asarray(amplitudes).ravel()
    x = x[np.isfinite(x)]
    if x.size == 0:
        raise ValueError("No finite amplitude values to plot.")

    # Freedman–Diaconis binning (robust)
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3 - q1
    if iqr <= 0:
        nbins = int(np.clip(np.sqrt(x.size), 10, 150))
    else:
        h = 2 * iqr * (x.size ** (-1/3))
        nbins = int(np.clip(np.ceil((x.max() - x.min()) / max(h, 1e-12)), 20, 150))

    # Log-bins if positive & wide dynamic range
    use_log = log_bins and np.all(x > 0) and (x.max() / max(x.min(), 1e-12) >= 100)
    bins = (np.logspace(np.log10(x.min()), np.log10(x.max()), nbins)
            if use_log else nbins)

    # Plot
    ax.hist(x, bins=bins, density=density, edgecolor="black", linewidth=0.6)
    if use_log:
        ax.set_xscale("log")
        ax.set_xlabel("Amplitude (log scale)")
    else:
        ax.set_xlabel("Amplitude (counts per 0.5 ms)")

    ax.set_ylabel("Density" if density else "Count")
    ax.set_title(title, pad=8, fontsize=12)

    # Clean look
    ax.yaxis.grid(True, linestyle="-", linewidth=0.5, alpha=0.35)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    # Summary stats
    mu, med = float(np.mean(x)), float(np.median(x))
    ax.axvline(mu, linestyle="--", linewidth=1.0, label=f"Mean = {mu:.3g}")
    ax.axvline(med, linestyle=":",  linewidth=1.0, label=f"Median = {med:.3g}")
    ax.legend(frameon=False, loc="upper left")


# ---------- Prepare data for the left (seaborn) plot ----------
sns.set_theme(style="whitegrid", context="paper")

cats = [-1, 1, 2, 3]
labels = {-1: "Power Law", 1: "1 species log-normal", 2: "2 species log-normal", 3: "3 species log-normal"}

counts = np.array([(types == c).sum() for c in cats], dtype=float)
total = counts.sum()
proportions = counts / total if total > 0 else np.zeros_like(counts)

df = pd.DataFrame({
    "Type": [labels[c] for c in cats],
    "Proportion": proportions,
    "Count": counts.astype(int)
})

# ---------- Combined figure ----------
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12.5, 4.2), dpi=300, constrained_layout=True)

# Left: seaborn bar (proportions)
sns.barplot(data=df, x="Type", y="Proportion", ax=ax1)
ax1.set_xlabel("")
ax1.set_ylabel("Percentage")
ax1.yaxis.set_major_formatter(PercentFormatter(xmax=1))
ax1.set_title("Fluorescence Distribution Percentages", pad=8, fontsize=12)
ax1.yaxis.grid(True, linestyle="-", linewidth=0.5, alpha=0.35)
sns.despine(ax=ax1, top=True, right=True)
ax1.tick_params(axis="x", rotation=12)

ymax = df["Proportion"].max() if len(df) else 0
ax1.set_ylim(0, min(1.0, ymax * 1.18 + 0.02))

for p, n, patch in zip(df["Proportion"], df["Count"], ax1.patches):
    h = patch.get_height()
    ax1.annotate(f"{p*100:.1f}%\n(n={n})",
                 (patch.get_x() + patch.get_width()/2, h),
                 ha="center", va="bottom", fontsize=9)

# Right: histogram of amplitudes
publication_hist_ax(ax2, amplitudes, title="Average Amplitude Distribution", log_bins=False, density=False)

# Save & show
fig.savefig("combined_types_amplitudes.svg", dpi=1000, bbox_inches="tight")
fig.savefig("combined_types_amplitudes.png", dpi=1000, bbox_inches="tight")
plt.show()
